In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [64]:
def ConvertDateintoDay_Month_Year(df):
    newdf = pd.DataFrame()
    df.Date = pd.to_datetime(df.Date)
    newdf["Nav"] = df.Nav
    newdf["day"] = df.Date.dt.day
    newdf["month"] = df.Date.dt.month
    newdf["year"] = df.Date.dt.year
    return newdf
def OptimiseDataframe(df):
    newdf = pd.DataFrame()
    newdf["Date"] = df.Date
    newdf["Nav"] = df.Open
    newdf.dropna()
    return newdf
def PredictBestMutualFund(day,month,year,duration,algo):
    start = []
    end = []
    ans_start = []
    ans_end = []
    ans = []
    m = duration[0]
    y = duration[1]
    
    ele = (month+m)//12
    m = (month+m)%13
    y =y+ele+year
    
    input_start = [[day,month,year]]*len(algo)
    input_end = [[day,m,y]]*len(algo)
    for i in range(len(algo)):
        x = input_start[i].copy()
        start.append(x)
        y = input_end[i].copy()
        end.append(y)
    
    for i in range(len(start)):
        x = algo[i].predict([start[i]])
        y = algo[i].predict([end[i]])
        ans_start.append(x[0])
        ans_end.append(y[0])
        ans.append(CalculateReturn(x[0],y[0]))
        
    print(start,"\n\n",end)
    print("-------------------------")
    print(ans_start,"\n\n",ans_end)
    print("-------------------------")
    new_ans = [ans_end[i]-ans_start[i] for i in range(len(ans_start))]
    print(new_ans)
    print("-------------------------")
    return ans

In [3]:
def CalculateReturn(start,end):
    returns = (end-start)/start
    returns = returns*100
    return returns

In [6]:
dir_ = os.getcwd()
dir_ = os.path.join(dir_,"dataset")
dir_ = os.listdir(dir_)
csv_file = [ele for ele in dir_ if ".csv" in ele]

l = []
directory =  os.path.join(os.getcwd(),"dataset")
for i in csv_file:
    df = pd.read_csv(os.path.join(directory,i))
    l.append(df)
    
df_list = []
for i in range(len(l)):
    newdf = OptimiseDataframe(l[i])
    newdf = newdf.dropna()
    df_list.append(newdf)

In [12]:
new_df_list =[]
for i in range(len(df_list)):
    new_df = ConvertDateintoDay_Month_Year(df_list[i])
    new_df_list.append(new_df)

In [49]:
lr_model_list = []
for i in range(len(new_df_list)):
    y = new_df_list[i]["Nav"]
    X = new_df_list[i].drop(["Nav"],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    regr = LinearRegression()
    regr.fit(X_train, y_train)
    print("LR Model "+str(i)+" : ",regr.score(X_test, y_test))
    lr_model_list.append(regr)
    

LR Model 0 :  0.8985593288089925
LR Model 1 :  0.908807336959529
LR Model 2 :  0.8978374961970118
LR Model 3 :  0.9014599475882349
LR Model 4 :  0.902876227193987
LR Model 5 :  0.9127670432060478
LR Model 6 :  0.9073168221410625
LR Model 7 :  0.8971509692044437
LR Model 8 :  0.8981952880357563
LR Model 9 :  0.888369645744664


In [50]:
ans_list = PredictBestMutualFund(23,2,2023,[5,1],lr_model_list)
print(ans_list)
print(max(ans_list))

[[23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023], [23, 2, 2023]] 

 [[23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024], [23, 7, 2024]]
-------------------------
[138.62955336333835, 139.34195179474045, 140.1411949489775, 139.87889195886964, 139.22260675265716, 139.04767636106408, 139.55812149549456, 138.27635750498303, 139.78252852162404, 138.77472023609153] 

 [180.03797129348095, 181.5508792805049, 182.67593367787777, 181.66153103001125, 180.60144839408167, 180.89749237403885, 181.8351355988998, 179.95423551449494, 181.76594342408498, 180.44133394242817]
-------------------------
[41.408417930142605, 42.208927485764434, 42.534738728900265, 41.7826390711416, 41.378841641424515, 41.84981601297477, 42.277014103405236, 41.677878009511915, 41.983414902460936, 41.66661370633665]
-------------------------
[29.86983433583894

In [46]:
def get_max_return_fund(l):
    max_return = 0
    index = -1
    for i in range(len(l)):
        if l[i]>max_return:
            max_return = l[i]
            index = i
    return [max_return,index]

In [51]:
print(get_max_return_fund(ans_list))

[30.351345829744268, 2]


In [52]:
print(csv_file[get_max_return_fund(ans_list)[1]])

SBI Equity Hybrid Fund.csv


In [85]:
import pickle

for i in range(len(lr_model_list)):
    name = csv_file[i]
    name = name.split(".")[0]
    name = name+" model.pkl"
    file = open(name, 'wb')
    # dump information to that file
    pickle.dump(lr_model_list[i], file)

In [38]:
csv_file

['sbi consumption opportunities fund.csv',
 'sbi contra fund.csv',
 'SBI Equity Hybrid Fund.csv',
 'sbi focused fund.csv',
 'sbi healthcare opportunities fund.csv',
 'SBI Large & Midcap Fund.csv',
 'SBI Magnum Equity ESG Fund.csv',
 'SBI Magnum Income Fund.csv',
 'SBI Nifty Index Fund.csv',
 'sbi small cap.csv']

In [43]:
rnd_fr_model_list = []
for i in range(len(new_df_list)):
    y = new_df_list[i]["Nav"]
    X = new_df_list[i].drop(["Nav"],axis=1)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)
    regr = RandomForestRegressor()
    regr.fit(X_train, y_train)
    print("LR Model "+str(i)+" : ",regr.score(X_test, y_test))
    rnd_fr_model_list.append(regr)

LR Model 0 :  0.9969995265514571
LR Model 1 :  0.9984541552058925
LR Model 2 :  0.9971751976506348
LR Model 3 :  0.9983824789106962
LR Model 4 :  0.9984202517680367
LR Model 5 :  0.9986689831144804
LR Model 6 :  0.9984704362759232
LR Model 7 :  0.998685288888299
LR Model 8 :  0.9983964153576629
LR Model 9 :  0.9982961783474464


In [84]:
ans_list = PredictBestMutualFund(23,11,2021,[8,0],rnd_fr_model_list)
print(ans_list)

[[23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021], [23, 11, 2021]] 

 [[23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022], [23, 6, 2022]]
-------------------------
[115.1459281199999, 115.47326637999996, 115.38734226999985, 115.69810163999989, 115.38443829999986, 115.27916125999985, 115.44161836999993, 115.41226355999984, 115.39811426999984, 115.43279629999982] 

 [105.49484558000003, 106.02612061999987, 106.4176715600001, 105.42165014000007, 106.37299470999994, 105.05026923999992, 105.59251043000009, 105.06703760999993, 105.51785529000003, 105.24322234000009]
-------------------------
[-9.651082539999862, -9.447145760000083, -8.969670709999747, -10.276451499999823, -9.011443589999914, -10.228892019999932, -9.84910793999984, -10.345225949999914, -9.880258979999809, -10.189573959999734]
-------------------------


In [68]:
print(get_max_return_fund(ans_list))
print(csv_file[get_max_return_fund(ans_list)[1]])

[256.3870417359191, 2]
SBI Equity Hybrid Fund.csv


In [91]:
lr_model_list[9].predict([[23,12,2019]])
file = open("SBI small cap model.pkl", 'wb')
# dump information to that file
pickle.dump(lr_model_list[9], file)



# 